# [로지스틱회귀분석 실습 1: 이진분류 문제]

## 1. 모듈 불러오기

In [ ]:
from IPython.display import display, HTML

# 데이터 전처리
import numpy as np
import pandas as pd

# 기계학습 모델
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

## Universal bank 데이터
#### 설명변수 및 반응변수
- ID - 고객 고유번호 <br>
- Age - 고객의 연령 (연속형) <br>
- Experienc - 고객의 경력 (연속형) <br>
- Incom - 고객의 연간 수입 (연속형) <br>
- ZIP Code - 고객 주소 ZIP Code <br>
- Family - 고객의 가족 규모 (연속형) <br>
- CCAvg - 평균 신용 카드 지출 (연속형) <br>
- Education - 교육 수준 (범주형 - 1: 대학교 졸업, 2: 대학원 졸업, 3: 고급/전문) <br>
- Mortgag - 주택 모기지의 가치 (연속형) <br>
- PersonalLoan - 고객은 마지막 캠페인에 제공된 개인 대출을 수락 하였나? (범주형 - 0: 아니오, 1: 예) <b>[타겟 변수]</b> <br>
- SecuritiesAccount - 고객은 은행에 증권 계좌를 가지고 있는가? (범주형 - 0: 아니오, 1: 예) <br>
- CDAccount - 고객은 은행에 예금 계좌를 가지고 있는가?	(범주형 - 0: 아니오, 1: 예) <br>
- Online - 고객이 인터넷 뱅킹 시설을 사용하는가? (범주형 - 0: 아니오, 1: 예) <br>
- CreditCard - 고객이 Universal Bank에서 발행 한 신용 카드를 사용하는가? (범주형 - 0: 아니오, 1: 예)

In [ ]:
data = pd.read_csv('Data/UniversalBank.csv')
data.head()

## 2. 데이터 전처리 및 탐색적 데이터 분석

### ID와 ZIP Code는 예측에 필요하지 않은 변수이므로 제거

In [ ]:
data = data.drop(['ID','ZIP Code'], axis=1)
data.head()

In [ ]:
plt.figure(figsize = (8,8))
sns.pairplot(data, hue="Personal Loan")
plt.show()

### 나이, 수입, 평균 신용 카드 지출, 대출 여부만 가져와서 보자

In [ ]:
plt.figure(figsize = (8,8))
sns.pairplot(data[['Age', 'Income', 'CCAvg', 'Personal Loan']], hue="Personal Loan")
plt.show()

### Education 변수에 대한 더미 변수 생성
- Education - 교육 수준 (범주형 - 1: 대학교 졸업, 2: 대학원 졸업, 3: 고급/전문) 

In [ ]:
data['Education'].head(10)

In [ ]:
pd.get_dummies(data['Education']).head(5)

In [ ]:
pd.get_dummies(data['Education'], drop_first=True).head(10)

In [ ]:
education_dummy = pd.get_dummies(data['Education'], drop_first=True)
education_dummy.columns = ['Edu_Dummy_1', 'Edu_Dummy_2']
display(education_dummy.head())

### 기존 Education 변수 제거하고, 더미 변수 추가

In [ ]:
data = data.drop('Education', axis=1)
data.head()

In [ ]:
data = pd.concat([data, education_dummy], axis=1)
data.head()

### 타겟 변수의 클래스 비율 확인

In [ ]:
display(data['Personal Loan'].value_counts())

### 타겟 변수의 클래스 비율 유지하며 Train/Test 데이터 분리
- 설명변수(X), 반응변수(y) 나누기

In [ ]:
X = data.drop('Personal Loan', axis=1)
X.head(10)

In [ ]:
y = data['Personal Loan']
y.head(10)

- Training 데이터 70% / Testing 데이터 30% 나누기
- 클래스 비율 유지: train_test_split 함수 내 stratify 옵션

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=20190625, stratify=y)

In [ ]:
print('클래스별 데이터 개수: Training 데이터  ')
display(y_train.value_counts())
print('클래스별 데이터 개수: Testing')
display(y_test.value_counts())

## 3. 모델링
- statsmodels(sm) 의 logistic 회귀모델 (Logistic) 함수를 이용하자.
- model = sm.logit(y데이터,x데이터)

In [ ]:
model = sm.Logit(y_train, X_train)
model_fitted = model.fit(method='newton')

## 4. 모델 결과물 확인

In [ ]:
model_fitted.summary()

#### coef 확인

In [ ]:
coef= model_fitted.params
print(coef)

#### odds ratio 확인

In [ ]:
np.exp(coef)

## 5. 모델 예측 성능 구하기

### 로지스틱회귀분석 모델의 출력 값은 "클래스 1에 속할 확률"

- training 데이터 예측성능을 평가해보자

In [ ]:
train_prob = model_fitted.predict(X_train)
train_prob.head()

In [ ]:
train_prob = model_fitted.predict(X_train)
train_prob.head(10)

In [ ]:
train_prob = model_fitted.predict(X_train)
train_results = pd.concat([train_prob, y_train], axis=1)
train_results.columns = ['Predicted Probability of Class 1', 'Personal Loan']
display(train_results)

- Testing 데이터 예측성능을 평가해보자

In [ ]:
test_prob = model_fitted.predict(X_test)
test_results = pd.concat([test_prob, y_test], axis=1)
test_results.columns = ['Predicted Probability of Class 1', 'Personal Loan']
display(test_results)

### 확률 값을 기준으로 최종 클래스 결정 (0.5를 기준으로)

In [ ]:
train_class = train_prob.copy()
test_class = test_prob.copy()

In [ ]:
train_class[train_class > 0.5] = 1
train_class[train_class <= 0.5] = 0

test_class[test_class > 0.5] = 1
test_class[test_class <= 0.5] = 0

In [ ]:
train_class.head()

In [ ]:
test_class.head()

### Train 데이터에 대한 정확도

In [ ]:
accuracy_score(y_train, train_class)

### Test 데이터에 대한 정확도

In [ ]:
accuracy_score(y_test, test_class)

## 6. Classification 결과 평가

In [ ]:
confusion_matrix(y_train, train_class)

In [ ]:
display(pd.DataFrame(confusion_matrix(y_train, train_class), columns=[0,1], index=[0,1]))
print('Training Accuracy: {:.3f}'.format(accuracy_score(y_train, train_class)))

In [ ]:
display(pd.DataFrame(confusion_matrix(y_test, test_class), columns=[0,1], index=[0,1]))
print('Testing Accuracy: {:.3f}'.format(accuracy_score(y_test, test_class)))

## 7. 클래스 판단 기준 바꿔보기

In [ ]:
train_class_2 = train_prob.copy()
test_class_2 = test_prob.copy()

train_class_2[train_class_2 > 0.3] = 1
train_class_2[train_class_2 <= 0.3] = 0

test_class_2[test_class_2 > 0.3] = 1
test_class_2[test_class_2 <= 0.3] = 0

print('Train Accuracy: {:.3f}'.format(accuracy_score(y_train, train_class_2)))
display(pd.DataFrame(confusion_matrix(y_train, train_class_2), columns=[0,1], index=[0,1]))
print('Test Accuracy: {:.3f}'.format(accuracy_score(y_test, test_class_2)))
display(pd.DataFrame(confusion_matrix(y_test, test_class_2), columns=[0,1], index=[0,1]))

In [ ]:
train_class_3 = train_prob.copy()
test_class_3 = test_prob.copy()

train_class_3[train_class_3 > 0.3] = 1
train_class_3[train_class_3 <= 0.3] = 0

test_class_3[test_class_3 > 0.3] = 1
test_class_3[test_class_3 <= 0.3] = 0

print('Train Accuracy: {:.3f}'.format(accuracy_score(y_train, train_class_3)))
display(pd.DataFrame(confusion_matrix(y_train, train_class_3), columns=[0,1], index=[0,1]))
print('Test Accuracy: {:.3f}'.format(accuracy_score(y_test, test_class_3)))
display(pd.DataFrame(confusion_matrix(y_test, test_class_3), columns=[0,1], index=[0,1]))

## 8. Classification 결과 평가 심화

In [ ]:
import pycm

train_class_3 = train_prob.copy()
test_class_3 = test_prob.copy()

train_class_3[train_class_3 > 0.3] = 1
train_class_3[train_class_3 <= 0.3] = 0

test_class_3[test_class_3 > 0.3] = 1
test_class_3[test_class_3 <= 0.3] = 0

## array 형태로 적용, type 맞춰주기
cm = pycm.ConfusionMatrix(y_test.values.astype(int), test_class_3.values.astype(int))

In [ ]:
print(cm)

https://en.wikipedia.org/wiki/Confusion_matrix

https://en.wikipedia.org/wiki/Evaluation_of_binary_classifiers